# 네이버 뉴스 크롤러

### 본 크롤러는 네이버 뉴스 크롤러 입니다.

## ========== ver 0.4 =========

#### - <u>세부 파라미터값이 있습니다. 함수에서 직접 조절해주세요.</u>

## ========== ver 0.3 =========

#### - naver.news 개편에 따라 크롤링 태그 변경이 있었습니다. (update by. daesun97)
#### - 전처리 기능을 더욱 강화했습니다. (update by. daesun97)
#### - 기사 카테고리 분류가 추가되었습니다.
#### - 기간내 특정 일자들의 기사를 놓치는 문제를 해결하고자 알고리즘 수정이 있었습니다.
#### - 자동으로 검색 일자를 변경해 최소 1주일 단위로 기사를 수집합니다.

## ========== ver 0.2 =========

#### - 본문의 전처리가 추가되었습니다. (update by. daesun97)

## ========== ver 0.1 =========
#### - 본 크롤러는 안내에 맞게 날짜를 입력해주셔야 합니다.
#### - 처음 사용하시는 분이라면 시작 전 본인의 컴퓨터에 !pip install fake_useragent 를 꼭 해주세요.
#### - fake_useragent로 ip차단걱정을 덜었습니다.
#### - 가급적 로컬 환경에서 사용해주세요. (colab 환경에서 실험을 아직 안해봤습니다)
#### - excel로 자동 저장됩니다.
#### - 각종 이슈 발생시 꼭 알려주세요.




###### (제작자 : suhykii)
###### 참조 블로그 리스트


In [1]:
#!pip install fake_useragent

  Using cached fake-useragent-0.1.11.tar.gz (13 kB)
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=69bf3d1f29f7c36f6f258f1d049fbba3339878220979c0884413be8ba2ee3ffe
  Stored in directory: c:\users\zls99\appdata\local\pip\cache\wheels\ed\f7\62\50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [9]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import csv

In [26]:
## 크롤링 함수
def crawler(keyword, start_day, last_day):
    df = pd.DataFrame(columns=['title', 'category', 'date', 'paper', 'url', 'content'])

    last_day = pd.to_datetime(last_day)
    front_day = pd.to_datetime(start_day)
    back_day = pd.to_datetime(start_day) + timedelta(days=6)
    new_len=0
    while (last_day >= back_day) & (last_day >= front_day):
        front_day = str(front_day)[:10]
        back_day = str(back_day)[:10]
        s_d_f = front_day.replace("-",".")
        e_d_f = back_day.replace("-",".")
        s_d_b = front_day.replace("-","")
        e_d_b = back_day.replace("-","")
        page=1
        less_news_cnt = 0
        old_len = new_len
        while less_news_cnt < 8: #이 숫자가 커질수록 기사를 많이 긁어와요. 조절을 잘 해야해요.
            url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}&sort=0&photo=0&field=0&pd=3&ds={s_d_f}&de={e_d_f}&cluster_rank=47&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{s_d_b}to{e_d_b},a:all&start="+str(page)

            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')

            for urls in soup.select("a.info"):
                naver_news_cnt = len([urls for urls in soup.select("a.info") if urls["href"].startswith("https://news.naver.com") ])
                if naver_news_cnt < 4: # 이 숫자가 작을수록 기사를 많이 긁어와요. 조절을 잘해야해요.
                    less_news_cnt+=1

                try:
                    if urls["href"].startswith("https://news.naver.com"):
                        news_detail = []

                        ua=UserAgent()
                        headers = {"User-Agent" : ua.random}

                        breq = requests.get(urls["href"], headers = headers)
                        bsoup = BeautifulSoup(breq.content, 'html.parser')

                        title = bsoup.select('h2.media_end_head_headline')[0].text
                        news_detail.append(title)

                        category = bsoup.select('.media_end_categorize_item')[0].text
                        news_detail.append(category)

                        date = bsoup.select('._ARTICLE_DATE_TIME')[0].text[:10]
                        date = date.replace(".","")
                        news_detail.append(date)

                        pcompany = bsoup.select('.media_end_linked_more_point')[0].text
                        news_detail.append(pcompany)

                        news_detail.append(urls["href"])

                        _text = bsoup.select('#newsct_article')[0].get_text().replace('\n', "")
                        btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
                        btext=btext[:btext.find(('▶'))]
                        btext=btext[:btext.find(('▷'))]
                        btext=re.sub('[【<\[].*?[>\]】]', '', btext)
                        btext=re.sub('[◆©ⓒ■◇]', '', btext)
                        btext=re.sub('[a-zA-Z0-9+-_.]+@[a-zA-Z0-9]+\.[a-zA-Z0-9-.]+\.*[a-z]*','',btext)
                        btext=re.sub('[0-9]+\.[0-9]*\.[0-9]*','',btext)
                        btext=re.sub('동+영+상','',btext)
                        btext=re.sub('[뉴]+[스]','',btext)
                        btext=re.sub('사+진+\=','',btext)
                        btext=re.sub('\(+사+진+=+.*?\)','',btext)
                        btext=re.sub('\(+[서]+[울]+=+[연]+[합]+\)','',btext)
                        btext=re.sub('....[기]+[자]+.=','',btext)
                        btext=re.sub('...[기]+[자]','',btext)
                        btext=re.sub('연+합+뉴+스','',btext)
                        btext=re.sub('\(+\)+네+이+버+.*','',btext)
                        btext=re.sub('.+제+공+\=','',btext)

                        news_detail.append(btext.strip())

                        df.loc[len(df)] = news_detail
                except Exception as e:
                    pass


            page += 10
        new_len = len(df)
        print(f"{front_day}부터 {back_day}까지 기사를",new_len-old_len,"개 추가했습니다.")
        front_day = pd.to_datetime(front_day) + timedelta(days=7)
        back_day = pd.to_datetime(back_day) + timedelta(days=7)
        if back_day >= last_day:
            back_day = last_day
    return df

def main():
    info_main = input("="*50+"\n"+"시작전에 상단에 설명을 잘 읽어주세요"+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    keyword = input("="*50+"\n"+"검색하실 키워드는 어떻게 되나요?"+"\n"+"정확하게 작성해주세요"+"\n"+"="*50)
    start_day = input("="*50+"\n"+"언제부터 시작할까요?"+"\n"+"반드시 0000-00-00 형식으로 작성해주세요"+"\n"+"="*50)
    last_day = input("="*50+"\n"+"언제까지 탐색할까요?"+"\n"+"반드시 0000-00-00 형식으로 작성해주세요"+"\n"+"="*50)
    print("="*40+"크롤링을 시작합니다"+"="*40)
    #rate = int(input("="*50+"\n"+"대략 몇개의 기사가 출력되길 원하시나요?"+"\n"+"10의 자리수로 끊어서 숫자만"+"\n"+"최대 5000"+"\n"+"="*50))//10
    s_d_b = start_day.replace("-","")[2:]
    e_d_b = last_day.replace("-","")[2:]
    df = crawler(keyword, start_day, last_day)
    
    df.to_excel(f'{keyword}_{s_d_b}~{e_d_b} 테스트 데이터.xlsx',index=False,encoding='utf-8-sig')
    print("\n")
    print(f"현재 폴더에 {keyword}_{s_d_b}~{e_d_b} 테스트 데이터.xlsx 이름으로 저장했습니다.")

    return df

### 종목명, 기사 기간,  기사 최대 개수를 적어주세요. 
#### 완료되면 자동으로 excel을 생성합니다. (노트 위치에 저장)
#### 입력하신 종목, 날짜를 기준으로 파일명이 저장됩니다.
#### "df" 변수로 저장되어있어 바로 확인 가능합니다.

In [27]:
%%time
df = main()

시작전에 상단에 설명을 잘 읽어주세요
 시작하시려면 Enter를 눌러주세요.
검색하실 키워드는 어떻게 되나요?
정확하게 작성해주세요
==================================================네이버
언제부터 시작할까요?
반드시 0000-00-00 형식으로 작성해주세요
==================================================2017-01-01
언제까지 탐색할까요?
반드시 0000-00-00 형식으로 작성해주세요
==================================================2022-03-31
========================================크롤링을 시작합니다========================================
2017-01-01부터 2017-01-07까지 기사를 41 개 추가했습니다.
2017-01-08부터 2017-01-14까지 기사를 33 개 추가했습니다.
2017-01-15부터 2017-01-21까지 기사를 28 개 추가했습니다.
2017-01-22부터 2017-01-28까지 기사를 153 개 추가했습니다.
2017-01-29부터 2017-02-04까지 기사를 32 개 추가했습니다.
2017-02-05부터 2017-02-11까지 기사를 33 개 추가했습니다.
2017-02-12부터 2017-02-18까지 기사를 21 개 추가했습니다.
2017-02-19부터 2017-02-25까지 기사를 44 개 추가했습니다.
2017-02-26부터 2017-03-04까지 기사를 84 개 추가했습니다.
2017-03-05부터 2017-03-11까지 기사를 33 개 추가했습니다.
2017-03-12부터 2017-03-18까지 기사를 79 개 추가했습니다.
2017-03-19부터 2017-03-25까지 기사를 115 개 추가했습니다.
2017-03-26부터 2017-04-01까지 기사를 41 개 추가했습니다.
2017-04-02부터 2017-04

2020-05-10부터 2020-05-16까지 기사를 17 개 추가했습니다.
2020-05-17부터 2020-05-23까지 기사를 76 개 추가했습니다.
2020-05-24부터 2020-05-30까지 기사를 18 개 추가했습니다.
2020-05-31부터 2020-06-06까지 기사를 81 개 추가했습니다.
2020-06-07부터 2020-06-13까지 기사를 54 개 추가했습니다.
2020-06-14부터 2020-06-20까지 기사를 41 개 추가했습니다.
2020-06-21부터 2020-06-27까지 기사를 84 개 추가했습니다.
2020-06-28부터 2020-07-04까지 기사를 138 개 추가했습니다.
2020-07-05부터 2020-07-11까지 기사를 211 개 추가했습니다.
2020-07-12부터 2020-07-18까지 기사를 132 개 추가했습니다.
2020-07-19부터 2020-07-25까지 기사를 304 개 추가했습니다.
2020-07-26부터 2020-08-01까지 기사를 356 개 추가했습니다.
2020-08-02부터 2020-08-08까지 기사를 85 개 추가했습니다.
2020-08-09부터 2020-08-15까지 기사를 56 개 추가했습니다.
2020-08-16부터 2020-08-22까지 기사를 42 개 추가했습니다.
2020-08-23부터 2020-08-29까지 기사를 20 개 추가했습니다.
2020-08-30부터 2020-09-05까지 기사를 23 개 추가했습니다.
2020-09-06부터 2020-09-12까지 기사를 250 개 추가했습니다.
2020-09-13부터 2020-09-19까지 기사를 34 개 추가했습니다.
2020-09-20부터 2020-09-26까지 기사를 128 개 추가했습니다.
2020-09-27부터 2020-10-03까지 기사를 127 개 추가했습니다.
2020-10-04부터 2020-10-10까지 기사를 433 개 추가했습니다.
2020-10-11부터 2020-10-17까지 기사를 316 개 추가했습니다.
2

In [30]:
 len(df["category"]=="경제")

23175

In [31]:
len(df)

23175

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23175 entries, 0 to 23174
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     23175 non-null  object
 1   category  23175 non-null  object
 2   date      23175 non-null  object
 3   paper     23175 non-null  object
 4   url       23175 non-null  object
 5   content   23175 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [35]:
df["category"].value_counts()

IT      11992
경제       7889
사회       1518
정치       1016
생활        568
오피니언      115
세계         77
Name: category, dtype: int64

In [36]:
df["paper"].value_counts()

연합뉴스       1737
뉴시스        1724
이데일리       1574
뉴스1        1522
한국경제       1182
           ... 
앳스타일          2
코메디닷컴         1
뉴스타파          1
kbc광주방송       1
국제신문          1
Name: paper, Length: 81, dtype: int64

#### 기사가 집중된 특정 기간에 대한 확인이 필요하니 기간+기사갯수가 있는 df가 따로 있으면 좋을듯함
#### 주간단위를 끊어서 가져가는 것도 괜찮을듯
#### 월별기사를 따로 정리하는 함수를 삽입하는 것도 괜찮을듯..
#### 주가데이터 상승,하락이 모두 만들어지면 연동해서 EDA하는 것도 괜찮을듯.. 날짜가 주요 골자가 되겠지